In [ ]:
import pandas as pd
import tweepy
import time
from ttictoc import tic,toc
import pickle
import csv
import json
pd.set_option('display.max_colwidth', None)


In [ ]:
df = pd.read_csv('200 Benign Accounts.csv')
df2 = pd.read_csv('400 benign account.csv')

In [ ]:
with open('Benign_Account_138.data', 'rb') as filehandle:
        x= pickle.load( filehandle)

In [ ]:
x[-1].user

In [ ]:
dfx = [df, df2]

In [ ]:
dfx = pd.concat(dfx)

In [ ]:
dfx = dfx.drop_duplicates('TwitterScreenName')

In [ ]:
dfx = [df, dfx]
dfx = pd.concat(dfx)

In [ ]:
dfx = dfx.drop_duplicates(subset='TwitterScreenName', keep=False)

In [ ]:
df=dfx

In [ ]:
df

In [ ]:
screen_name = df['TwitterScreenName']


In [ ]:
screen_name=screen_name[160:]

In [ ]:
screen_name = df['TwitterScreenName']

In [ ]:
screen_name = screen_name.drop_duplicates(keep='last')

In [ ]:
screen_name

In [ ]:
key1 = ""
key2 = ""
key3 = ""
key4 = ""


auth = tweepy.OAuthHandler(key1, key2)
auth.set_access_token(key3, key4)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
it = 0
old_length = 0
new_length =0
consumer_key = key1
consumer_secret = key2
access_key = key3
access_secret = key4


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

for i in screen_name:
    old_length = len(alltweets)
    try:
        new_tweets = api.user_timeline(screen_name = i,count=200, tweet_mode="extended")

        alltweets.extend(new_tweets)

        oldest = alltweets[-1].id - 1

        while len(new_tweets) > 0:

            new_tweets = api.user_timeline(screen_name = i,count=200,max_id=oldest, tweet_mode="extended")

            alltweets.extend(new_tweets)

            oldest = alltweets[-1].id - 1
            with open('Benign_Account_'+str(it) +'.data', 'wb') as filehandle:
                pickle.dump(alltweets, filehandle)

        new_length = len(alltweets) - old_length
        print(f"...{new_length} tweets downloaded so far for {i}")
        time.sleep(15*30)
        it+=1
        
    except:
        print("Account Not Found: " + str(i))



In [ ]:
tic()


api = tweepy.API(auth, wait_on_rate_limit=True, retry_count=10, retry_delay=300, retry_errors=set([503]))

for accountvar in screen_name:
    try:
        followerids =[]

        for user in tweepy.Cursor(api.followers_ids, screen_name=accountvar,count=5000).items():
            followerids.append(user)
            time.sleep(.0002)
        print (len(followerids))

        with open('U:/Twitter Research/Fake News Scraping/Fake News Account Followers Data/Follower_ID' 
                  + str(accountvar) + '.data', 'wb') as filehandle:
            pickle.dump(followerids, filehandle)

        print(toc())
    except:
        print(str(accountvar) + " not in Twitter.")



In [ ]:
for accountvar in screen_name:
    print (accountvar)

In [ ]:
api = tweepy.API(auth, wait_on_rate_limit=True, retry_count=10, retry_delay=300, retry_errors=set([503]))
followerids =[]

In [ ]:
tic()
accountvar = ''
followerids =[]

for user in tweepy.Cursor(api.get_follower_ids, screen_name=accountvar,count=5000).items():
    followerids.append(user)
    time.sleep(.0002)
print (len(followerids))

with open('U:/Twitter Research/Fake News Scraping/Fake News Account Followers Data/Follower_ID/' 
          + str(accountvar) + '.data', 'wb') as filehandle:
    pickle.dump(followerids, filehandle)

print(toc())


In [ ]:
len(followerids)

In [ ]:
userids = followerids

In [ ]:
api = tweepy.API(auth, wait_on_rate_limit=True, retry_count=10, retry_delay=300, retry_errors=set([503]))


accountvar = ""
tic()
def get_usernames(userids, api):
    fullusers = []
    u_count = len(userids)
    print(u_count)
    try:
        for i in range(int(u_count/100) + 1):            
            end_loc = min((i + 1) * 100, u_count)
            fullusers.extend(
                api.lookup_users(user_id=userids[i * 100:end_loc])                
            )
            if(len(fullusers) % 20000 < 99):
                print(len(fullusers) % 20000)
                print(len(fullusers))
                time.sleep(150)
        return fullusers
    except:
        import traceback
        traceback.print_exc()
        print ('Something went wrong, quitting...')

fullusers = get_usernames(followerids,api)
print(toc())


with open('U:/Twitter Research/Fake News Scraping/Fake News Account Followers Data/Follower_Info' 
                  + str(accountvar) + '.data', 'wb') as filehandle:
    pickle.dump(fullusers, filehandle)

In [ ]:
len(fullusers)

In [ ]:
df_link = pd.read_excel('Fake Tweet Link.xlsx')

In [ ]:
df_link = df_link.dropna()

In [ ]:
df_link

In [ ]:
def link_to_id(link):
    return str(link).split('/')[-1]

df_link['Tweet_ID'] = df_link['Link'].apply(link_to_id)

In [ ]:
def valid_link(id):
    res = True
    if (len(id) < 5) or id =='Post Deleted':
        res = False
    
    return res

df_link['Valid_Link'] = df_link['Tweet_ID'].apply(valid_link)    

In [ ]:
df_link

In [ ]:
df_link.loc[df_link.Valid_Link ==False]

In [ ]:
df_link.loc[df_link.Valid_Link == True]

In [ ]:
len(df_link)

In [ ]:
k = 0
for index, row in df_link.iterrows():
    if(row['Valid_Link'] == True):
        try:
            rt = api.get_retweets(row['Tweet_ID'], count = 100)
            print(len(rt))
            with open('U:/Twitter Research/Fake News Scraping/Fake News Tweet Sharing/RT_UPDATED/RT_' 
                  + str(row['Tweet_ID']) + '.data', 'wb') as filehandle:
                pickle.dump(rt, filehandle)
            time.sleep(60)
        except:
            print(str(row['Tweet_ID']) + ' -  tweet got deleted')

In [ ]:
fav = client.get_liking_users()

In [ ]:
k = 0
for index, row in df_link.iterrows():
    if(row['Valid_Link'] == True):
        try:
            fav = client.get_liking_users(row['Tweet_ID'])
            fav_id =[] 
            i=0
            for i in fav.data:
                fav_id.append(i.id)
            with open('U:/Twitter Research/Fake News Scraping/Fake News Tweet Sharing/LIKE/LIKE_' 
                      + str(row['Tweet_ID']) + '.data', 'wb') as filehandle:
                pickle.dump(fav_id, filehandle)
            time.sleep(60)
            print('Success for : ' + str(row['Tweet_ID']))
        except:
            print(str(row['Tweet_ID']) + ' -  tweet got deleted')

In [ ]:
type(fav)

In [ ]:
fav.data[0].id

In [ ]:
with open('your_data.json', 'w') as out:
    json.dump(fav.data[0].id,out)